In [1]:
from hlda.sampler import HierarchicalLDA
import pandas as pd

data = pd.read_csv("matrix_data.csv",header=None)
pd_vocab = pd.read_csv("words_dict.csv",header=None)
idx = data.sum()==1
validcol = [col for i,col in enumerate(data.columns) if not idx[i] ]
validwords = [word for i,word in enumerate(list(pd_vocab[0])) if not idx[i] ]
reduced_data = data[validcol]
reduced_data.columns = range(reduced_data.shape[1])

In [2]:
new_corpus = []
#index is the row index, row is the document 
for index, row in reduced_data.iterrows():
    new_doc = []
    for i in range(len(validwords)):
        if row[i]!= 0:
            for j in range(row[i]):
                new_doc.append(i)
    new_corpus.append(new_doc)

In [3]:
n_samples = 500       # no of iterations for the sampler
alpha = 10.0          # smoothing over level distributions
gamma = 1.0           # CRP smoothing parameter; number of imaginary customers at next, as yet unused table
eta = 0.1             # smoothing over topic-word distributions
num_levels = 2        # the number of levels in the tree
display_topics = 50   # the number of iterations between printing a brief summary of the topics so far
n_words = 10          # the number of most probable words to print for each topic after model estimation
with_weights = False  # whether to print the words with the weights

In [5]:
hlda = HierarchicalLDA(new_corpus, validwords, alpha=alpha, gamma=gamma, eta=eta, num_levels=num_levels)
hlda.estimate(n_samples, display_topics=display_topics, n_words=n_words, with_weights=with_weights)

HierarchicalLDA sampling
.................................................. 50
topic 0 (level=0, total_words=71757, documents=14640): flight, not, thank, get, no, can, just, will, now, time, 
    topic 1 (level=1, total_words=3519, documents=636): hour, delay, plane, flight, gate, sit, wait, late, connect, min, 
    topic 2 (level=1, total_words=4004, documents=728): hold, call, hour, phone, tri, help, get, can, onlin, reserv, 
    topic 3 (level=1, total_words=2139, documents=417): email, can, number, pleas, book, dm, phone, confirm, not, form, 
    topic 4 (level=1, total_words=2081, documents=413): servic, custom, worst, ever, airlin, cancel, never, call, terribl, poor, 
    topic 5 (level=1, total_words=1938, documents=379): cancel, flight, flightl, hotel, delay, rebook, get, miss, airport, flightlat, 
    topic 6 (level=1, total_words=2183, documents=417): cancel, flight, hold, flightl, hour, get, late, wait, tomorrow, help, 
    topic 7 (level=1, total_words=667, documents=145): 

KeyboardInterrupt: 

In [94]:
doc_topic_res = hlda.document_leaves
doc_topic_res[0]
all_group = []
size_check = 0
for n in range(145):
    each_group = [ i for i in range(14640) if doc_topic_res[i].node_id == n]
    size_check += len(each_group)
    all_group.append(each_group)
size_check

14576

In [91]:
doc_topic_res[2265]

Node=146 level=1 customers=17 total_words=42 parent=0

In [79]:
total_set = range(14640)

In [99]:
topic_set = [i for group in all_group for i in group]
all_group.append(list(set(total_set) - set(topic_set)))

In [100]:
new_groups = all_group[1:]
## get the true labels here 
len(new_groups)
#len(doc_topic_res)
# True Lables 
#new_groups[142]


145

In [107]:
Origin_data = pd.read_csv("Tweets.csv")
True_label = Origin_data["airline_sentiment"]

# get the label for each group 
label_array = []
correct_num = 0
for groups in new_groups:
    pos_sum = sum([ 1 for value in True_label[groups].values if value == "positive"] )
    neu_sum = sum([ 1 for value in True_label[groups].values if value == "neutral"] )
    neg_sum = sum([ 1 for value in True_label[groups].values if value == "negative"] )
    temp_sum = [pos_sum,neu_sum,neg_sum]
    if pos_sum == max(temp_sum):
        label_array.append("positive")
    if neu_sum == max(temp_sum):
        label_array.append("neutral")
    if neg_sum == max(temp_sum):
        label_array.append("negative")
    correct_num += max(temp_sum)
print(len(label_array))
print(float(correct_num) / 14640 )

145
0.660245901639


In [102]:
##### get the matrix result for the hlda
file_matrix = []
for n in range(len(new_groups)):
    for group in new_groups[n]:
        file_matrix.append([group,label_array[n]])

In [105]:
file_label = pd.DataFrame(file_matrix)
file_label.shape
#tempdf = file_label.sort_values(by=[0])
#tempdf.to_csv("hlda_result.csv", index=False)

(14640, 2)